In [1]:
#프로젝트의 루트 디렉토리를 설정하기 위한 코드
import os
current_dir = os.getcwd()  # 현재 작업 디렉토리
print(os.path.dirname(current_dir))
# print(os.path.dirname(os.path.dirname(current_dir)))
# 루트 경로 설정 (예: 프로젝트 루트 경로)
project_root = os.path.dirname(current_dir)
os.chdir(project_root)  # 루트 경로로 변경

c:\sb-fsts\sb-fsts


In [2]:
import datetime
import matplotlib.pyplot as plt
import mplfinance as mpf
import pandas as pd
import numpy as np
import requests

In [ ]:
from app.utils.auto_trading_bot import AutoTradingBot
from datetime import date, time


id = "id1"

auto_trading_stock = AutoTradingBot(id=id, virtual=False)

# symbol = '329180'
symbol = '012330'

start_date = date(2023, 1, 1)
end_date = date(2023, 12, 10)

target_trade_value_krw = 1000000  # 매수 목표 거래 금액
buy_trading_logic = ['stochastic_trading']
sell_trading_logic = ['stochastic_trading']
interval = 'day'

# ✅ 시뮬레이션 실행 (df 내부 값을 확인할 수 있음)

auto_trading_stock.simulate_trading(
    symbol=symbol,
    start_date=start_date,
    end_date=end_date,
    target_trade_value_krw=target_trade_value_krw,
    buy_trading_logic=buy_trading_logic,
    sell_trading_logic=sell_trading_logic,
    interval=interval,
    buy_percentage=None,
    ohlc_mode='default',
    initial_capital=None
)



실전투자 API 객체가 성공적으로 생성되었습니다.
stochastic_k : 2023-01-02 00:00:00+09:00   NaN
Name: stochastic_k, dtype: float64
stochastic_d : 2023-01-02 00:00:00+09:00   NaN
Name: stochastic_d, dtype: float64
❌ MFI 계산을 위한 데이터가 부족합니다.
투자비용: 0
❌ MFI 계산을 위한 데이터가 부족합니다.
총 비용: 0KRW, 총 보유량: 0주, 평균 단가: 0KRW, 실현 손익 (Realized PnL): 0KRW, 미실현 손익 (Unrealized PnL): 0KRW
stochastic_k : 2023-01-02 00:00:00+09:00   NaN
2023-01-03 00:00:00+09:00   NaN
Name: stochastic_k, dtype: float64
stochastic_d : 2023-01-02 00:00:00+09:00   NaN
2023-01-03 00:00:00+09:00   NaN
Name: stochastic_d, dtype: float64
투자비용: 0
총 비용: 0KRW, 총 보유량: 0주, 평균 단가: 0KRW, 실현 손익 (Realized PnL): 0KRW, 미실현 손익 (Unrealized PnL): 0KRW
stochastic_k : 2023-01-02 00:00:00+09:00   NaN
2023-01-03 00:00:00+09:00   NaN
2023-01-04 00:00:00+09:00   NaN
Name: stochastic_k, dtype: float64
stochastic_d : 2023-01-02 00:00:00+09:00   NaN
2023-01-03 00:00:00+09:00   NaN
2023-01-04 00:00:00+09:00   NaN
Name: stochastic_d, dtype: float64
투자비용: 0
총 비용: 0KRW, 총 보유량: 0주, 평균 

(                                 Time      Open      High       Low     Close  \
 2023-01-02 00:00:00+09:00  2023-01-02  201000.0  206500.0  201000.0  202500.0   
 2023-01-03 00:00:00+09:00  2023-01-03  201500.0  205500.0  197500.0  202000.0   
 2023-01-04 00:00:00+09:00  2023-01-04  201500.0  205500.0  200000.0  203000.0   
 2023-01-05 00:00:00+09:00  2023-01-05  202000.0  209000.0  202000.0  207000.0   
 2023-01-06 00:00:00+09:00  2023-01-06  204000.0  216500.0  204000.0  214000.0   
 ...                               ...       ...       ...       ...       ...   
 2023-12-04 00:00:00+09:00  2023-12-04  229000.0  229000.0  225500.0  225500.0   
 2023-12-05 00:00:00+09:00  2023-12-05  225500.0  226500.0  224000.0  225000.0   
 2023-12-06 00:00:00+09:00  2023-12-06  225000.0  227500.0  223500.0  223500.0   
 2023-12-07 00:00:00+09:00  2023-12-07  223000.0  226000.0  220500.0  224500.0   
 2023-12-08 00:00:00+09:00  2023-12-08  226000.0  227500.0  224500.0  226500.0   
 
              

In [ ]:
auto_trading_stock._get_ohlc(symbol, start_date, end_date)
